Nível Infra:

* Salvar o dataset em uma bucket, no Cloud Storage.
* Conceder acesso aos professores.
* Salvar os dataframes também na em uma bucket, no Cloud Storage.


Preparando o ambiente:

In [ ]:
#INSTALANDO FERRAMENTAS NECESSÁRIAS: PYSPARK
!pip install pyspark 

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=37a7a393b568810947c774ce8512ff1dca01e418e6c93524495ef6c111b48f00
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
#IMPORTANDO BIBLIOTECS NECESSÁRIAS: PANDAS, SPARKSESSION, PYSPARK.SQL, STRUCTTYPE E FUNCTIONS
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.window import Window


In [ ]:
#CRIANDO UMA CONEXÃO COM O SPARK:
spark = (SparkSession.builder
         .master("local")
         .appName("miniProjetoFinalSoulCode")
         .config("spark.ui.port", "4050")
         .getOrCreate())

In [ ]:
#TESTANDO A CONEXÃO RECÉM-CRIADA:
spark

Nível Pandas:    
 

In [ ]:
#EXTRAINDO O DATAFRAME
df = pd.read_csv("/content/drive/MyDrive/marketing_campaign.csv", sep="\t",parse_dates=["Dt_Customer"], dayfirst=True)
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,43,182,42,118,247,2,9,3,4,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,2014-06-10,56,406,0,30,0,0,8,7,8,2,5,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,48,217,32,12,24,1,2,3,13,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,30,214,80,30,61,2,6,5,10,3,0,0,0,0,0,0,3,11,0


In [ ]:
#CRIANDO UM BACKUP:
backup = df.copy()

In [ ]:
#DROPAR COLUNAS SE NECESSÁRIO

#Dropando a coluna "Z_CostContact", porque supondo que o objetivo do df seja buscar razões que levaram o cliente a executar uma compra, ela é descnecessária além de que seus dados são repetidos. 
df.drop(["Z_CostContact"], axis=1, inplace=True) 

#Dropando a coluna "Z_Revenue", pelo mesmo motivo da coluna anterior.
df.drop(["Z_Revenue"], axis=1, inplace=True) 


In [ ]:
#TRADUZINDO O DATAFRAME PARA PORTUGÊS:

#Traduzindo os campos para português
df.loc[df.Education == "Graduation", ["Education"]] = "Graduacao"
df.loc[df.Education == "PhD", ["Education"]] = "Doutorado"
df.loc[df.Education == "Basic", ["Education"]] = "Basica"
df.loc[df.Education == "2n Cycle", ["Education"]] = "2ª Graduacao"
df.loc[df.Education == "Master", ["Education"]] = "Mestrado"

df.loc[df.Marital_Status == "Single", ["Marital_Status"]] = "Solteiro"
df.loc[df.Marital_Status == "Alone", ["Marital_Status"]] = "Solteiro"
df.loc[df.Marital_Status == "Together", ["Marital_Status"]] = "Morando Junto"
df.loc[df.Marital_Status == "Married", ["Marital_Status"]] = "Casado"
df.loc[df.Marital_Status == "Divorced", ["Marital_Status"]] = "Divorciado"
df.loc[df.Marital_Status == "Widow", ["Marital_Status"]] = "Viuvo"
df.loc[df.Marital_Status == "Absurd", ["Marital_Status"]] = "Incerto"
df.loc[df.Marital_Status == "YOLO", ["Marital_Status"]] = "Noivo"

In [ ]:
#Traduzindo o cabeçalho para português
df.rename(columns={"ID": "idCliente",
                   "Year_Birth": "anoNascimento",
                   "Education": "escolaridade",
                   "Marital_Status": "statusMatrimonial",
                   "Income": "rendaAnual",
                   "Kidhome": "filhoCrianca",
                   "Teenhome": "filhoAdolesc",
                   "Dt_Customer": "ingressoCliente",
                   "Recency": "diasUltimaCompra",
                   "MntWines": "gastosVinho",
                   "MntFruits":	"gastosFrutas",
                   "MntMeatProducts":	"gastosCarne",
                   "MntFishProducts":	"gastosProdutos",
                   "MntSweetProducts": "gastosProdsDoces",
                   "MntGoldProds": "gastosProdsOuro",
                   "NumDealsPurchases":	"comprasDesconto",
                   "NumWebPurchases":	"comprasWeb",
                   "NumCatalogPurchases":	"comprasCatalogo",
                   "NumStorePurchases":	"comprasLoja",
                   "NumWebVisitsMonth":	"visitasWebMes",
                   "AcceptedCmp3": "ofertaAceitaCampanha3",
                   "AcceptedCmp4": "ofertaAceitaCampanha4",
                   "AcceptedCmp5": "ofertaAceitaCampanha5",
                   "AcceptedCmp1": "ofertaAceitaCampanha1",
                   "AcceptedCmp2": "ofertaAceitaCampanha2",
                   "Complain": "reclamacoes",
                   "Response":"ofertaAceitaUltimaCamp"}, inplace=True)


In [ ]:
#LIMPANDO DADOS INCONSISTENTES

#Convertendo dados incoerentes

tiposDeDados = {
       "escolaridade": "string",
       "statusMatrimonial": "string",
       "gastosVinho": "float64",
       "gastosFrutas": "float64",
       "gastosCarne": "float64",
       "gastosProdutos": "float64",
       "gastosProdsDoces": "float64",
       "gastosProdsOuro": "float64"
       }
  
df = df.astype(tiposDeDados)
df.dtypes


idCliente                          int64
anoNascimento                      int64
escolaridade                      string
statusMatrimonial                 string
rendaAnual                       float64
filhoCrianca                       int64
filhoAdolesc                       int64
ingressoCliente           datetime64[ns]
diasUltimaCompra                   int64
gastosVinho                      float64
gastosFrutas                     float64
gastosCarne                      float64
gastosProdutos                   float64
gastosProdsDoces                 float64
gastosProdsOuro                  float64
comprasDesconto                    int64
comprasWeb                         int64
comprasCatalogo                    int64
comprasLoja                        int64
visitasWebMes                      int64
ofertaAceitaCampanha3              int64
ofertaAceitaCampanha4              int64
ofertaAceitaCampanha5              int64
ofertaAceitaCampanha1              int64
ofertaAceitaCamp

In [ ]:
#SALVANDO O DATAFRAME RESULTANTE
df.to_csv("clear_marketing_campaign.csv", index=False)

Nível PySpark:

In [ ]:
#MONTANDO O DATAFRAME COM "STRUCTTYPE"

schema = StructType() \
      .add("idCliente",IntegerType(),True) \
      .add("anoNascimento",IntegerType(),True) \
      .add("escolaridade",StringType(),True) \
      .add("statusMatrimonial",StringType(),True) \
      .add("rendaAnual",DoubleType(),True) \
      .add("filhoCrianca",IntegerType(),True) \
      .add("filhoAdolesc",IntegerType(),True) \
      .add("ingressoCliente",DateType(),True) \
      .add("diasUltimaCompra",IntegerType(),True) \
      .add("gastosVinho",DoubleType(),True) \
      .add("gastosFrutas",DoubleType(),True) \
      .add("gastosCarne",DoubleType(),True) \
      .add("gastosProdutos",DoubleType(),True) \
      .add("gastosProdsDoces",DoubleType(),True) \
      .add("gastosProdsOuro",DoubleType(),True) \
      .add("comprasDesconto",IntegerType(),True) 

df1 = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/content/drive/MyDrive/clear_marketing_campaign.csv")


In [ ]:
df1.printSchema()
df1.show()

root
 |-- idCliente: integer (nullable = true)
 |-- anoNascimento: integer (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- statusMatrimonial: string (nullable = true)
 |-- rendaAnual: double (nullable = true)
 |-- filhoCrianca: integer (nullable = true)
 |-- filhoAdolesc: integer (nullable = true)
 |-- ingressoCliente: date (nullable = true)
 |-- diasUltimaCompra: integer (nullable = true)
 |-- gastosVinho: double (nullable = true)
 |-- gastosFrutas: double (nullable = true)
 |-- gastosCarne: double (nullable = true)
 |-- gastosProdutos: double (nullable = true)
 |-- gastosProdsDoces: double (nullable = true)
 |-- gastosProdsOuro: double (nullable = true)
 |-- comprasDesconto: integer (nullable = true)

+---------+-------------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+
|idCliente|anoNascimento|escolaridade|st

In [ ]:
#REALIZANDO UM BACKUP DO DATAFRAME
backup1 = df1

In [ ]:
#ALTERANDO O NOME DE DUAS COLUNAS

df1 = df1.withColumnRenamed("idCliente", "id").withColumnRenamed("anoNascimento", "anoNasc")
df1.show(1)


+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+
|  id|anoNasc|escolaridade|statusMatrimonial|rendaAnual|filhoCrianca|filhoAdolesc|ingressoCliente|diasUltimaCompra|gastosVinho|gastosFrutas|gastosCarne|gastosProdutos|gastosProdsDoces|gastosProdsOuro|comprasDesconto|
+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+
|5524|   1957|   Graduacao|         Solteiro|   58138.0|           0|           0|     2012-09-04|              58|      635.0|        88.0|      546.0|         172.0|            88.0|           88.0|              3|
+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+-----

In [ ]:
#CRIANDO DUAS NOVAS COLUNAS CONTENDO INFORMAÇÕES RELEVANTES SOBRE AS OUTRAS COLUNAS JÁ EXISTENTES

#Criando uma nova coluna com a soma dos gastos totais
df1 = df1.withColumn("gastosTotais", expr("gastosVinho + gastosFrutas + gastosCarne + gastosProdutos + gastosProdsDoces + gastosProdsOuro"))
df1.show(1)


+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+------------+
|  id|anoNasc|escolaridade|statusMatrimonial|rendaAnual|filhoCrianca|filhoAdolesc|ingressoCliente|diasUltimaCompra|gastosVinho|gastosFrutas|gastosCarne|gastosProdutos|gastosProdsDoces|gastosProdsOuro|comprasDesconto|gastosTotais|
+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+------------+
|5524|   1957|   Graduacao|         Solteiro|   58138.0|           0|           0|     2012-09-04|              58|      635.0|        88.0|      546.0|         172.0|            88.0|           88.0|              3|      1617.0|
+----+-------+------------+-----------------+----------+------------+-----------

In [ ]:
#Criando uma nova coluna que caso o cliente não realize compras há mais de dois meses, informe sua ausência
df1 = (df1.withColumn("ausente", F.when(F.col("diasUltimaCompra") > 60, "sim")
          .otherwise("nao"))    
      )
df1.show(10)

+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+------------+-------+
|  id|anoNasc|escolaridade|statusMatrimonial|rendaAnual|filhoCrianca|filhoAdolesc|ingressoCliente|diasUltimaCompra|gastosVinho|gastosFrutas|gastosCarne|gastosProdutos|gastosProdsDoces|gastosProdsOuro|comprasDesconto|gastosTotais|ausente|
+----+-------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+------------+-------+
|5524|   1957|   Graduacao|         Solteiro|   58138.0|           0|           0|     2012-09-04|              58|      635.0|        88.0|      546.0|         172.0|            88.0|           88.0|              3|      1617.0|    nao|
|2174|   1954|   Graduacao|         Solteiro|   

In [ ]:
#UTILIZANDO FILTROS, ORDENACAO E AGRUPAMENTO, PARA TRAZER DADOS RELEVANTES PARA O NEGÓCIO EM QUESTAO

#Somando os gastos totais por status civil, afim de descobrir quais deles gastam mais e com o quê.
df1.groupBy(F.col("statusMatrimonial")).agg(
    F.sum(F.col("gastosVinho")).alias("gastoTotalVin"),
    F.sum(F.col("gastosFrutas")).alias("gastoTotalFru"),
    F.sum(F.col("gastosCarne")).alias("gastoTotalCar"),
    F.sum(F.col("gastosProdutos")).alias("gastoTotalPro"),
    F.sum(F.col("gastosProdsDoces")).alias("gastoTotalPD"),
    F.sum(F.col("gastosProdsOuro")).alias("gastoTotalPO")
).orderBy("statusMatrimonial").show()

+-----------------+-------------+-------------+-------------+-------------+------------+------------+
|statusMatrimonial|gastoTotalVin|gastoTotalFru|gastoTotalCar|gastoTotalPro|gastoTotalPD|gastoTotalPO|
+-----------------+-------------+-------------+-------------+-------------+------------+------------+
|           Casado|     258751.0|      22235.0|     138829.0|      30569.0|     23070.0|     36999.0|
|       Divorciado|      75364.0|       6363.0|      34848.0|       8130.0|      6222.0|     10739.0|
|          Incerto|        711.0|        169.0|        625.0|        411.0|        61.0|       408.0|
|    Morando Junto|     177959.0|      14703.0|      97500.0|      22615.0|     15151.0|     24937.0|
|            Noivo|        644.0|          6.0|        100.0|          8.0|         6.0|        84.0|
|         Solteiro|     138953.0|      12893.0|      87491.0|      18367.0|     13107.0|     21071.0|
|            Viuvo|      28434.0|       2548.0|      14575.0|       3957.0|      3

In [ ]:
#Contando as compras totais por status civil, afim de descobrir quais deles realizam mais compras e quanto gastam em vinho
df1.groupBy(F.col("statusMatrimonial")).agg(
    F.count("statusMatrimonial").alias("quantCompras"),
    F.sum("gastosVinho").alias("gastoTotalVin"),
    F.sum("gastosTotais").alias("gastoTotal")
    ).orderBy(F.col("quantCompras").desc()).show()

+-----------------+------------+-------------+----------+
|statusMatrimonial|quantCompras|gastoTotalVin|gastoTotal|
+-----------------+------------+-------------+----------+
|           Casado|         864|     258751.0|  510453.0|
|    Morando Junto|         580|     177959.0|  352865.0|
|         Solteiro|         483|     138953.0|  291882.0|
|       Divorciado|         232|      75364.0|  141666.0|
|            Viuvo|          77|      28434.0|   56889.0|
|            Noivo|           2|        644.0|     848.0|
|          Incerto|           2|        711.0|    2385.0|
+-----------------+------------+-------------+----------+



In [ ]:
#Filtrando os maiores gastos com vinho e carne; Agrupando, contando e somando afim de descobrir em qual nível de escolaridade estão aqueles que mais gastam com vinho e carne. 
df_vin_car = df1.filter((F.col("gastosVinho") > 700) & (F.col("gastosCarne") > 500))

df_vin_car.groupBy(F.col("escolaridade")).agg(
    F.count(F.col("escolaridade")).alias("quantEscol"),
    F.sum(F.col("gastosVinho")).alias("somaGastosVinho"),
    F.sum(F.col("gastosCarne")).alias("somagGastosCarne")
).orderBy(F.col("quantEscol").desc()).show()


+------------+----------+---------------+----------------+
|escolaridade|quantEscol|somaGastosVinho|somagGastosCarne|
+------------+----------+---------------+----------------+
|   Graduacao|        38|        35034.0|         27155.0|
|   Doutorado|        29|        27345.0|         19222.0|
|    Mestrado|        25|        23932.0|         17877.0|
|2ª Graduacao|         2|         1809.0|          1458.0|
+------------+----------+---------------+----------------+



In [ ]:
#UTILIZANDO WINDOW FUNCTIONS

#Usando o window function para descobrir quais clientes estão mais ausentes, ordenando em ordem decrescente
w0 = Window.orderBy(F.col("diasUltimaCompra").desc())

(df1.select(col("id"), col("gastosTotais"), col("diasUltimaCompra"))
                   .withColumn("maiorAusencia", F.row_number()
                   .over(w0))
                   .show()
                   )

+-----+------------+----------------+-------------+
|   id|gastosTotais|diasUltimaCompra|maiorAusencia|
+-----+------------+----------------+-------------+
| 7829|        65.0|              99|            1|
| 8595|        67.0|              99|            2|
|  868|        65.0|              99|            3|
| 2106|        34.0|              99|            4|
|   22|       309.0|              99|            5|
| 7232|        67.0|              99|            6|
|10142|       689.0|              99|            7|
| 9977|       734.0|              99|            8|
| 7212|        65.0|              99|            9|
|  528|      1383.0|              99|           10|
| 7947|        37.0|              99|           11|
| 5263|        55.0|              99|           12|
| 3363|        34.0|              99|           13|
| 4070|      1078.0|              99|           14|
| 2831|      1179.0|              99|           15|
| 2415|       953.0|              99|           16|
| 1743|     

In [ ]:
#Criando um ranking por grau de escolaridade, para descobrir o status civil dos que mais gastam
w1 = Window.partitionBy(F.col("escolaridade")).orderBy(F.col("gastosTotais").desc())

(df1.select(col("id"), col("statusMatrimonial"), col("escolaridade"), col("rendaAnual"), col("gastosTotais"))
    .withColumn("ranking", F.row_number()
    .over(w1))
    .show()
)

+-----+-----------------+------------+----------+------------+-------+
|   id|statusMatrimonial|escolaridade|rendaAnual|gastosTotais|ranking|
+-----+-----------------+------------+----------+------------+-------+
| 2186|    Morando Junto|2ª Graduacao|   82326.0|      2257.0|      1|
| 4947|         Solteiro|2ª Graduacao|   89572.0|      2059.0|      2|
| 3139|         Solteiro|2ª Graduacao|   74116.0|      2052.0|      3|
| 4856|           Casado|2ª Graduacao|   82347.0|      1853.0|      4|
| 8722|           Casado|2ª Graduacao|   82347.0|      1853.0|      5|
| 2223|       Divorciado|2ª Graduacao|   64176.0|      1826.0|      6|
| 8720|    Morando Junto|2ª Graduacao|      null|      1679.0|      7|
| 5341|       Divorciado|2ª Graduacao|   81975.0|      1588.0|      8|
| 5989|       Divorciado|2ª Graduacao|   78353.0|      1576.0|      9|
| 7313|            Viuvo|2ª Graduacao|   74859.0|      1565.0|     10|
| 6050|           Casado|2ª Graduacao|   57036.0|      1513.0|     11|
| 3661

In [ ]:
#ANÁLISES

#Os clientes que mais gastam em uma compra são os solteiros.
#Os clientes que mais voltam as compras são os casados, mesmo não sendo os que mais gastam.
#Os clientes que mais gastam em vinho e carne, são os que estão se graduando.
#Os clientes que mais estão ausentes, gastam mais com vinho e carne
#Os clientes que mais gastam, idependente do nível de escolaridade, são os solteiros.

In [ ]:
#SALVANDO O DATAFRAME

(df1.write.format("csv")
    .option("header", "true")
    .option("delimiter",",")
    .save("/content/drive/MyDrive/new/struct_marketing_campaign.csv")
)

Nível Spark SQL:

In [ ]:
#EXPORTANDO O DATAFRAME RESULTANTE DO "NÍVEL PANDAS"
df5 = (spark
       .read
       .format("csv")
       .option("header", "true")
       .option("inferschema", "true")
       .option("delimiter", ",")
       .load("/content/drive/MyDrive/clear_marketing_campaign.csv")
       .createOrReplaceTempView("mkt_camp")
)

In [ ]:
#VISUALIZANDO A VIEW RECÉM CRIADA
spark.sql("""SELECT * FROM mkt_camp""").show()

+---------+-------------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+------------+-----------+--------------+----------------+---------------+---------------+----------+---------------+-----------+-------------+---------------------+---------------------+---------------------+---------------------+---------------------+-----------+----------------------+
|idCliente|anoNascimento|escolaridade|statusMatrimonial|rendaAnual|filhoCrianca|filhoAdolesc|ingressoCliente|diasUltimaCompra|gastosVinho|gastosFrutas|gastosCarne|gastosProdutos|gastosProdsDoces|gastosProdsOuro|comprasDesconto|comprasWeb|comprasCatalogo|comprasLoja|visitasWebMes|ofertaAceitaCampanha3|ofertaAceitaCampanha4|ofertaAceitaCampanha5|ofertaAceitaCampanha1|ofertaAceitaCampanha2|reclamacoes|ofertaAceitaUltimaCamp|
+---------+-------------+------------+-----------------+----------+------------+------------+---------------+----------------+-----------+----------

In [ ]:
#C1#

#Visualizando os gastos gerais dos clientes que sao casados, tem criança e adolescente em casa, e possuem renda anual de mais de R$60.000
spark.sql("""
  SELECT idCliente, gastosVinho, gastosFrutas, gastosCarne, gastosProdutos, gastosProdsDoces, gastosProdsOuro
  FROM mkt_camp
  WHERE statusMatrimonial == "Casado" and filhoCrianca > 0 and filhoAdolesc > 0 and rendaAnual > 60000
  ORDER BY rendaAnual DESC
""").show()

+---------+-----------+------------+-----------+--------------+----------------+---------------+
|idCliente|gastosVinho|gastosFrutas|gastosCarne|gastosProdutos|gastosProdsDoces|gastosProdsOuro|
+---------+-----------+------------+-----------+--------------+----------------+---------------+
|     9499|     1279.0|        15.0|      287.0|          20.0|             0.0|           15.0|
|     4786|      555.0|        82.0|      257.0|          93.0|            61.0|           61.0|
|     1081|      733.0|         9.0|      180.0|          12.0|            19.0|           66.0|
|     6566|      826.0|        50.0|      317.0|          50.0|            38.0|           38.0|
|     6131|      141.0|        70.0|      106.0|          72.0|            19.0|           23.0|
|    10660|      349.0|         7.0|       35.0|           0.0|             0.0|           47.0|
|      246|      199.0|         9.0|       57.0|          20.0|            21.0|            6.0|
|     9381|      328.0|       

In [ ]:
#C2#

#Visualizando a quantidade de clientes que mais gastam, de acordo com seus respectivos estado civis
spark.sql("""
  SELECT statusMatrimonial, 
    COUNT(idCliente) AS quantCli,
    SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro) AS totalGasto 
  FROM mkt_camp
  GROUP BY statusMatrimonial
  ORDER BY totalGasto
""").show()

+-----------------+--------+----------+
|statusMatrimonial|quantCli|totalGasto|
+-----------------+--------+----------+
|            Noivo|       2|     848.0|
|          Incerto|       2|    2385.0|
|            Viuvo|      77|   56889.0|
|       Divorciado|     232|  141666.0|
|         Solteiro|     483|  291882.0|
|    Morando Junto|     580|  352865.0|
|           Casado|     864|  510453.0|
+-----------------+--------+----------+



In [ ]:
#C3#

#Verificando o quanto costumam gastar os clientes que tem criança e adolescente em casa, e são solteiros
spark.sql("""
  SELECT idCliente, 
    SUM(filhoCrianca+filhoAdolesc) AS quantFilhos, 
    SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro) AS totalGasto
  FROM mkt_camp
  WHERE statusMatrimonial == "Solteiro"
  GROUP BY idCliente
  HAVING quantFilhos >= 2
  ORDER BY totalGasto DESC
""").show()

+---------+-----------+----------+
|idCliente|quantFilhos|totalGasto|
+---------+-----------+----------+
|     6749|          2|    1722.0|
|     4974|          3|    1392.0|
|     7966|          2|    1191.0|
|     8957|          2|    1120.0|
|     2863|          3|    1052.0|
|     8897|          2|     654.0|
|      433|          2|     632.0|
|    10379|          2|     574.0|
|     6887|          2|     564.0|
|     7500|          2|     564.0|
|     6097|          2|     561.0|
|     4426|          2|     527.0|
|     1165|          2|     496.0|
|    10971|          2|     473.0|
|     2552|          3|     446.0|
|     6798|          2|     433.0|
|     4168|          2|     433.0|
|     6237|          2|     416.0|
|     1406|          2|     415.0|
|     1542|          3|     396.0|
+---------+-----------+----------+
only showing top 20 rows



In [ ]:
#C4#

#Verificando o quanto costumam gastar os clientes que não tem criança e adolescente em casa e sao casados
spark.sql("""
  SELECT idCliente, 
    SUM(filhoCrianca+filhoAdolesc) AS quantFilhos, 
    SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro) AS totalGasto
  FROM mkt_camp
  WHERE statusMatrimonial == "Casado"
  GROUP BY idCliente
  HAVING quantFilhos == 0
  ORDER BY totalGasto DESC
""").show()

+---------+-----------+----------+
|idCliente|quantFilhos|totalGasto|
+---------+-----------+----------+
|     4580|          0|    2486.0|
|     4475|          0|    2440.0|
|     5453|          0|    2352.0|
|     9010|          0|    2346.0|
|      737|          0|    2231.0|
|     3334|          0|    2211.0|
|      477|          0|    2157.0|
|     7274|          0|    2153.0|
|     8867|          0|    2126.0|
|     2747|          0|    2126.0|
|     6932|          0|    2119.0|
|     3690|          0|    2091.0|
|     7627|          0|    2086.0|
|     1772|          0|    2043.0|
|     1172|          0|    2034.0|
|     1127|          0|    1958.0|
|    10936|          0|    1947.0|
|     5547|          0|    1919.0|
|     4543|          0|    1919.0|
|     8362|          0|    1919.0|
+---------+-----------+----------+
only showing top 20 rows



In [ ]:
#C5#

#Verificando a quantidade de clientes e a media de sua renda anual, em determinado grau de escolaridade 
spark.sql("""
  SELECT escolaridade, 
    COUNT(idCliente) AS quantCli,
    ROUND(AVG(rendaAnual),2) AS meRendaAnuaL
  FROM mkt_camp
  GROUP BY escolaridade
  HAVING escolaridade IN ("Graduacao", "Mestrado", "Doutorado")
  ORDER BY meRendaAnual
""").show()

+------------+--------+------------+
|escolaridade|quantCli|meRendaAnuaL|
+------------+--------+------------+
|   Graduacao|    1127|    52720.37|
|    Mestrado|     370|    52917.53|
|   Doutorado|     486|    56145.31|
+------------+--------+------------+



In [ ]:
#C6#¨

#Visualizando o valor medio gasto por clientes de acordo com seu grau de escolaridade
spark.sql("""
  SELECT escolaridade, 
    ROUND(AVG(gastosVinho),2) AS meGastaVinho,
    ROUND(AVG(gastosFrutas),2) AS meGastaFrutas,
    ROUND(AVG(gastosCarne),2) AS meGastaCarne,
    ROUND(AVG(gastosProdutos),2) AS meGastaProd,
    ROUND(AVG(gastosProdsDoces),2) AS meGastaProDoces,
    ROUND(AVG(gastosProdsOuro),2) AS meGastaProOuro
  FROM mkt_camp
  GROUP BY escolaridade
  ORDER BY escolaridade
""").show()

+------------+------------+-------------+------------+-----------+---------------+--------------+
|escolaridade|meGastaVinho|meGastaFrutas|meGastaCarne|meGastaProd|meGastaProDoces|meGastaProOuro|
+------------+------------+-------------+------------+-----------+---------------+--------------+
|2ª Graduacao|      198.18|        28.96|      141.26|      47.48|          34.25|          46.4|
|      Basica|        7.24|        11.11|       11.44|      17.06|          12.11|         22.83|
|   Doutorado|       404.5|        20.05|       168.6|      26.73|          20.22|         32.31|
|   Graduacao|      284.27|        30.77|      179.49|      43.15|          31.37|         50.85|
|    Mestrado|      333.08|        21.65|      163.38|       32.1|          21.18|          40.4|
+------------+------------+-------------+------------+-----------+---------------+--------------+



In [ ]:
#C7#

#Visualizando a fatura total obtida dos clientes que ingressaram nos anos de 2012, 2013 e 2014
spark.sql("""
  SELECT SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro) AS mnt_total_2012_13_14
  FROM mkt_camp
  WHERE ingressoCliente LIKE '%2012%'
  UNION
  SELECT SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro)
  FROM mkt_camp
  WHERE ingressoCliente LIKE '%2013%'
  UNION
  SELECT SUM(gastosVinho+gastosFrutas+gastosCarne+gastosProdutos+gastosProdsDoces+gastosProdsOuro) 
  FROM mkt_camp
  WHERE ingressoCliente LIKE '%2014%'
""").show()

+--------------------+
|mnt_total_2012_13_14|
+--------------------+
|            368269.0|
|            715425.0|
|            273294.0|
+--------------------+



In [ ]:
#C8#

#Verificando a quantidade total vendas obtidas nos diferentes tipos de comércio, por clientes que ingressaram em 2012
spark.sql("""
  SELECT SUM(comprasWeb) As totalCompWeb, 
    SUM(comprasCatalogo) AS totalCompCat, 
    SUM(comprasLoja) AS totalCompLoj
  FROM mkt_camp
  WHERE ingressoCliente LIKE '%2012%'
""").show()

+------------+------------+------------+
|totalCompWeb|totalCompCat|totalCompLoj|
+------------+------------+------------+
|        2334|        1486|        3049|
+------------+------------+------------+



In [ ]:
#C9#

#Vizualizando a quantidade total de compras realizadas nos diferentes tipos de comércio, por clientes solteiros e se graduando 
spark.sql("""
  SELECT SUM(comprasWeb) As totalCompWeb, 
    SUM(comprasCatalogo) AS totalCompCat, 
    SUM(comprasLoja) AS totalCompLoj
  FROM mkt_camp
  WHERE statusMatrimonial = "Solteiro"
  GROUP BY escolaridade
  HAVING escolaridade == "Graduacao"
""").show()

+------------+------------+------------+
|totalCompWeb|totalCompCat|totalCompLoj|
+------------+------------+------------+
|        1007|         688|        1430|
+------------+------------+------------+



In [ ]:
#C10#

#Vizualizando quantos clientes dos diferentes tipos de situacao matrimonial, que realizaram compras com desconto, mais visitam a página web
spark.sql("""
  SELECT statusMatrimonial, 
    COUNT(idCliente) AS quantCli, 
    SUM(visitasWebMes) AS quantVisitasWeb
  FROM mkt_camp
  WHERE comprasDesconto > 0
  GROUP BY statusMatrimonial
  ORDER BY quantVisitasWeb
""").show()

+-----------------+--------+---------------+
|statusMatrimonial|quantCli|quantVisitasWeb|
+-----------------+--------+---------------+
|          Incerto|       2|              3|
|            Noivo|       2|             16|
|            Viuvo|      74|            372|
|       Divorciado|     227|           1254|
|         Solteiro|     468|           2498|
|    Morando Junto|     567|           3010|
|           Casado|     854|           4611|
+-----------------+--------+---------------+

